In [1]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

from yolov4.tf import YOLOv4
import numpy as np
import cv2
import time

# Loading model and weights

In [2]:
yolo = YOLOv4()
yolo.classes = "test/coco.names"
yolo.make_model()
yolo.load_weights("weights/yolov4.weights", weights_type="yolo")

# Pre-processing 

In [3]:
# Used to pre-process images before used as input to the model.
def pre_process(img):
    resized_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resized_image = yolo.resize_image(resized_image)
    resized_image = resized_image / 255
    input_data = resized_image[np.newaxis, ...].astype(np.float32)
    
    return input_data

# Post-processing 

In [9]:
# Converts the model outputs to bounding boxes (BBs), and adds the BBs to the image.
def add_BBs(img, candidates, display=False):
    start_time = time.time()
    _candidates = []
    for candidate in candidates:
        batch_size = candidate.shape[0]
        grid_size = candidate.shape[1]
        _candidates.append(
            tf.reshape(
                candidate, shape=(1, grid_size * grid_size * 3, -1)
            )
        )
    # candidates == Dim(batch, candidates, (bbox))
    candidates = np.concatenate(_candidates, axis=1)

    # pred_bboxes == Dim(candidates, (x, y, w, h, class_id, prob))
    pred_bboxes = yolo.candidates_to_pred_bboxes(candidates[0], 0.3, 0.25)
    pred_bboxes = yolo.fit_pred_bboxes_to_original(
        pred_bboxes, img.shape
    )
    exec_time = time.time() - start_time
#     print("time: {:.2f} ms".format(exec_time * 1000))

    result = yolo.draw_bboxes(img, pred_bboxes)
    cv2.imwrite("result_example.jpg", result)
    
    if display:
        while True:
            cv2.imshow("result", result)

            if cv2.waitKey(1) == ord("q"):
                break
        
        cv2.destroyAllWindows()
        
    return result

# Loading test image 

In [10]:
original_image = cv2.imread("test/motorway.jpg")
# original_image = cv2.imread("test/kite.jpg")
input_data = pre_process(original_image)

# Predict using original way

In [11]:
start_time = time.time()
candidates = yolo.model.predict(input_data)
print(time.time() - start_time)

result = add_BBs(original_image, candidates, display=True)

0.14959979057312012


# Saving/Loading model

In [ ]:
# Save model for faster processing.
model = yolo.model
save_path = "saved_model"
tf.saved_model.save(model, save_path)

In [ ]:
# Load the saved model.
save_path = "saved_model"
st = time.time()
imported = tf.saved_model.load(save_path)
print("Model loaded after ", time.time() - st, "seconds.")

tf_model = imported.signatures["serving_default"]

# Predict using loaded model

In [ ]:
# Predict using the saved model.
def loaded_predict(processed_img):
    batch_data = tf.constant(processed_img)
    
#     st = time.time()
    candidates = tf_model(batch_data)
#     print(time.time() - st)
    
#     st = time.time()
    candidates = (candidates["output_1"].numpy(), candidates["output_2"].numpy(), candidates["output_3"].numpy())
#     print(time.time() - st)
    
    return candidates

In [ ]:
start_time = time.time()
candidates = loaded_predict(input_data)
print(time.time() - start_time)

result = add_BBs(original_image, candidates, display=True)

# Cam test (ensure a webcam is available)

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    input_data = pre_process(frame)
    
    st = time.time()
#     candidates = yolo.model.predict(input_data)
    candidates = loaded_predict(input_data)
    st = time.time()
    frame = add_BBs(frame, candidates)
    
    delay = round(time.time() - st, 4)
    
    
    cv2.putText(frame, str(delay), (40,40), 1, 2, (0, 0, 255), 3)
    cv2.imshow("frame", frame)
    
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()